In [1]:
import math
import numpy as np
import pandas as pd
from task import Task
from component import Component
from core import Core
from system import System
from simulator import Simulator

## Read csv

In [2]:
small_test_arch_path = "../DRTS_Project-Test-Cases/2-small-test-case/architecture.csv"
small_test_budgets_path = "../DRTS_Project-Test-Cases/2-small-test-case/budgets.csv"
small_test_tasks_path = "../DRTS_Project-Test-Cases/2-small-test-case/tasks.csv"

## Test System class

In [3]:
small_system = System(tasks_file=small_test_tasks_path, arch_file=small_test_arch_path, budgets_file=small_test_budgets_path)
small_system.tasks, small_system.components, small_system.cores

loaded dataframes: 
  core_id  speed_factor scheduler
0  Core_1          0.62       EDF
      component_id scheduler  budget  period core_id  priority
0    Camera_Sensor        RM       4       7  Core_1       NaN
1  Image_Processor       EDF       5      16  Core_1       NaN
  task_name  wcet  period     component_id  priority
0    Task_0     3     150    Camera_Sensor       1.0
1    Task_1    28     200    Camera_Sensor       2.0
2    Task_2     2      50    Camera_Sensor       0.0
3    Task_3    24     300    Camera_Sensor       3.0
4    Task_4     2     200  Image_Processor       NaN
5    Task_5    11     200  Image_Processor       NaN
6    Task_6    17     400  Image_Processor       NaN
7    Task_7    13     300  Image_Processor       NaN
8    Task_8     3     150  Image_Processor       NaN


({'Task_0': <task.Task at 0x7297c529c310>,
  'Task_1': <task.Task at 0x7297c529c940>,
  'Task_2': <task.Task at 0x7297c529cfd0>,
  'Task_3': <task.Task at 0x729784c5a950>,
  'Task_4': <task.Task at 0x7297c529d5d0>,
  'Task_5': <task.Task at 0x729784c5a4d0>,
  'Task_6': <task.Task at 0x729784c5b580>,
  'Task_7': <task.Task at 0x729784c5ace0>,
  'Task_8': <task.Task at 0x729784c5a9e0>},
 {'Camera_Sensor': <component.Component at 0x7297c529e230>,
  'Image_Processor': <component.Component at 0x7297c529d510>},
 {'Core_1': <core.Core at 0x7297c529cf10>})

In [4]:
def half_half_algorithm(budget: float, period: float):
    """
    Implements the Half-Half Algorithm for real-time systems.
    This algorithm calculates two parameters, alpha and delta, based on
    the given budget and period. These parameters can be used for
    scheduling or resource allocation in real-time systems.
    Parameters:
    -----------
    budget : float
        The allocated execution time or computational budget.
    period : float
        The time period over which the budget is allocated.
    Returns:
    --------
    alpha : represents the utilization factor, calculated as the ratio
        of budget to period.
    delta : represents a derived parameter, calculated as twice the
        difference between the period and the budget.
    """
    alpha = budget / period
    delta = 2 * (period - budget)
    return alpha, delta


half_half_algorithm(budget=4, period=7)

(0.5714285714285714, 6)

In [5]:
for core_name, core in small_system.cores.items():
    print(core_name)
    for component in core.components:
        print(component)

Core_1
Component(id=Camera_Sensor, scheduler=RM, budget=4, period=7, core_id=Core_1, priority=0.14285714285714285, tasks=4, bdr_alpha=0.57, bdr_delta=6.00)
Component(id=Image_Processor, scheduler=EDF, budget=5, period=16, core_id=Core_1, priority=0.0625, tasks=5, bdr_alpha=0.31, bdr_delta=22.00)


In [6]:
for component_name, component in small_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task)

Camera_Sensor
task Task_0 with wcet:3, period:150, component_id:Camera_Sensor, priority1.0
task Task_1 with wcet:28, period:200, component_id:Camera_Sensor, priority2.0
task Task_2 with wcet:2, period:50, component_id:Camera_Sensor, priority0.0
task Task_3 with wcet:24, period:300, component_id:Camera_Sensor, priority3.0
Image_Processor
task Task_4 with wcet:2, period:200, component_id:Image_Processor, priority0.005
task Task_5 with wcet:11, period:200, component_id:Image_Processor, priority0.005
task Task_6 with wcet:17, period:400, component_id:Image_Processor, priority0.0025
task Task_7 with wcet:13, period:300, component_id:Image_Processor, priority0.0033333333333333335
task Task_8 with wcet:3, period:150, component_id:Image_Processor, priority0.006666666666666667


## Test simulator class

In [7]:
small_simulator = Simulator(small_system)

In [8]:
small_simulator.run_simulation()

Running simulation until hyperperiod: 1200
Active components at time 0: ['Camera_Sensor', 'Image_Processor']
Scheduled components at time 0: ['Camera_Sensor', 'Image_Processor']
Processing component: Component(id=Camera_Sensor, scheduler=RM, budget=4, period=7, core_id=Core_1, priority=0.14285714285714285, tasks=4, bdr_alpha=0.57, bdr_delta=6.00)
Component Camera_Sensor: period is given by bdr_delta / 2 = 3.0, budget = 1.7142857142857142
Task task Task_0 with wcet:3, period:150, component_id:Camera_Sensor, priority1.0 release=0, deadline=150, now=3.0
Initialized WCET for Task task Task_0 with wcet:3, period:150, component_id:Camera_Sensor, priority1.0, instance 0: 4.838709677419355
Added Task task Task_0 with wcet:3, period:150, component_id:Camera_Sensor, priority1.0, instance 0 to active_tasks
Task task Task_1 with wcet:28, period:200, component_id:Camera_Sensor, priority2.0 release=0, deadline=200, now=3.0
Initialized WCET for Task task Task_1 with wcet:28, period:200, component_id:

In [9]:
for component_name, component in small_system.components.items():
    print(f"{component_name}")
    for task in component.tasks:
        print(task.response_times)

Camera_Sensor
[91.70967741935522]
[127.87096774193594]
[50.93548387096814, 10.161290322580953]
[107.70967741935478, 104.70967741935522]
Image_Processor
[177.38709677419388, 33.100806451613266, 109.44959677419388, 23.256048387097053]
[105.84274193548413, 193.19153225806485, 95.99798387096803]
[]
[292.12903225806485]
[18.87500000000034, 109.03024193548413]


In [10]:
small_simulator.report("results_small.csv")

task: task Task_0 with wcet:3, period:150, component_id:Camera_Sensor, priority1.0 is True with max_response_time91.70967741935522, component is True
task: task Task_1 with wcet:28, period:200, component_id:Camera_Sensor, priority2.0 is True with max_response_time127.87096774193594, component is True
task: task Task_2 with wcet:2, period:50, component_id:Camera_Sensor, priority0.0 is False with max_response_time50.93548387096814, component is False
task: task Task_3 with wcet:24, period:300, component_id:Camera_Sensor, priority3.0 is True with max_response_time107.70967741935478, component is False
task: task Task_4 with wcet:2, period:200, component_id:Image_Processor, priority0.005 is True with max_response_time177.38709677419388, component is True
task: task Task_5 with wcet:11, period:200, component_id:Image_Processor, priority0.005 is True with max_response_time193.19153225806485, component is True
task: task Task_6 with wcet:17, period:400, component_id:Image_Processor, priority0